<a href="https://colab.research.google.com/github/robgon-art/RockTopics/blob/main/2_RockTopics_Using_GPT_J_to_Get_Topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2. RockTopics - Using GPT-J to Get Topics**
## Sex and Drugs and Organic Topic Modeling - Using GPT-J to analyze the lyrics of Rock &amp; Roll songs

By Robert. A Gonsalves</br>
![topic graphs](https://raw.githubusercontent.com/robgon-art/RockTopics/main/rock_topics_cover_med.png)

You can see my article here on Medium.

The source code and datasets are released under the [CC BY-SA license](https://creativecommons.org/licenses/by-sa/4.0/).</br>
![CC BY-NC-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)

Based on [GPT-J-6B Inference Demo.ipynb](https://colab.research.google.com/github/kingoflolz/mesh-transformer-jax/blob/master/colab_demo.ipynb) by [@kingoflolz](https://github.com/kingoflolz)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown --id 112zGFLr2LsS9di7q2HhN2u2HK1q1FJ6I

Downloading...
From: https://drive.google.com/uc?id=112zGFLr2LsS9di7q2HhN2u2HK1q1FJ6I
To: /content/rock-lyrics.csv
100% 16.0M/16.0M [00:00<00:00, 97.7MB/s]


In [ ]:
import csv
rock_lyrics_file= open("rock-lyrics.csv")
rock_lyrics_csvreader = csv.reader(rock_lyrics_file)
rock_lyrics_header = next(rock_lyrics_csvreader)
print(rock_lyrics_header)

bands = []
rock_lyrics_data = []
for row in rock_lyrics_csvreader:
  band_name = row[rock_lyrics_header.index("Band Name")]
  if band_name not in bands:
    bands.append(band_name)
  rock_lyrics_data.append(row)

print("num bands", len(bands))
print("num lyrics", len(rock_lyrics_data))
print()

for b in bands:
  print(b)

['Band Name', 'Song', 'Lyrics']
num bands 50
num lyrics 11959

AC/DC
Aerosmith
Beastie Boys
Black Sabbath
Bob Dylan
Bruce Springsteen
Buddy Holly
Chuck Berry
Cream
Creedence Clearwater Revival
David Bowie
Elvis Costello
Elvis Presley
Eminem
Eric Clapton
Gram Parsons
Guns N' Roses
Janis Joplin
Jay-Z
Jerry Lee Lewis
Jimi Hendrix
John Lennon
Joni Mitchell
Led Zeppelin
Little Richard
Lynyrd Skynyrd
Madonna
Metallica
Michael Jackson
Neil Young
Nirvana
Patti Smith
Pink Floyd
Queen
R.E.M.
Radiohead
Rolling Stones
The Band
The Beach Boys
The Beatles
The Byrds
The Doors
The Police
The Who
Tina Turner
Tom Petty
Tupac Shakur
U2
Van Morrison
Velvet Underground


In [ ]:
!apt install zstd

# the "slim" version contain only bf16 weights and no optimizer parameters, which minimizes bandwidth and memory
# !time wget -c https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd
!time wget -c http://eaidata.bmk.sh/data/GPT-J-6B/step_383500_slim.tar.zstd


!time tar -I zstd -xf step_383500_slim.tar.zstd

!git clone https://github.com/kingoflolz/mesh-transformer-jax.git
!pip install -r mesh-transformer-jax/requirements.txt

# jax 0.2.12 is required due to a regression with xmap in 0.2.13
!pip install mesh-transformer-jax/ jax==0.2.12 tensorflow==2.5.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 278 kB of archives.
After this operation, 1,141 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 zstd amd64 1.3.3+dfsg-2ubuntu1.2 [278 kB]
Fetched 278 kB in 1s (385 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../zstd_1.3.3+dfsg-2ubuntu1.2_amd64.deb ...
Unpacking zstd (1.3.3+dfsg-2ubuntu1.2) ...
Setting up zstd (1.3.3+dfsg-2ubuntu1.2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
--2021-11-25 00:50:26--  http://eaidata.bmk.sh/data/GPT-J-6B/step_383500_slim.tar.zstd
Resolving eaidata.bmk.sh (eaidata.bmk.sh)... 176.9.113.70
Connecting to eaidata.bmk.sh (eaidata.bmk.sh)|176.9.113.70|:80... connec

Processing ./mesh-transformer-jax
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 590 kB 5.4 MB/s 
     |████████████████████████████████| 454.3 MB 15 kB/s 
     |████████████████████████████████| 4.0 MB 52.2 MB/s 
     |████████████████████████████████| 462 kB 69.2 MB/s 
     |████████████████████████████████| 1.2 MB 76.0 MB/s 
  Created wheel for mesh-transformer: filename=mesh_transformer-0.0.0-py3-none-any.whl size=26330 sha256=789ceaae67ef10c42b862d8ed17887754b093b2579c9aa9ccf1810733974a843
  Stored in directory: /root/.cache/pip/wheels/56/bd/89/b1f6b2f3d6b938d0c5812ee97756a1afd32521bea293543863
  Cr

## Setup Model


In [ ]:
import os
import requests 
from jax.config import config

colab_tpu_addr = os.environ['COLAB_TPU_ADDR'].split(':')[0]
url = f'http://{colab_tpu_addr}:8475/requestversion/tpu_driver0.1_dev20210607'
requests.post(url)

# The following is required to use TPU Driver as JAX's backend.
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']

Sometimes the next step errors for some reason, just run it again ¯\\\_(ツ)\_/¯

In [ ]:
import time

import jax
from jax.experimental import maps
import numpy as np
import optax
import transformers

from mesh_transformer.checkpoint import read_ckpt_lowmem
from mesh_transformer.sampling import nucleaus_sample
from mesh_transformer.transformer_shard import CausalTransformer

In [ ]:
params = {
  "layers": 28,
  "d_model": 4096,
  "n_heads": 16,
  "n_vocab": 50400,
  "norm": "layernorm",
  "pe": "rotary",
  "pe_rotary_dims": 64,

  "seq": 2048,
  "cores_per_replica": 8,
  "per_replica_batch": 1,
}

per_replica_batch = params["per_replica_batch"]
cores_per_replica = params["cores_per_replica"]
seq = params["seq"]


params["sampler"] = nucleaus_sample

# here we "remove" the optimizer parameters from the model (as we don't need them for inference)
params["optimizer"] = optax.scale(0)

mesh_shape = (jax.device_count() // cores_per_replica, cores_per_replica)
devices = np.array(jax.devices()).reshape(mesh_shape)

maps.thread_resources.env = maps.ResourceEnv(maps.Mesh(devices, ('dp', 'mp')))

tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')

Here we create the network and load the parameters from the downloaded files. Expect this to take around 5 minutes.

In [ ]:
total_batch = per_replica_batch * jax.device_count() // cores_per_replica

network = CausalTransformer(params)

network.state = read_ckpt_lowmem(network.state, "step_383500/", devices.shape[1])

network.state = network.move_xmap(network.state, np.zeros(cores_per_replica))

/usr/local/lib/python3.7/dist-packages/jax/experimental/maps.py:412: UserWarning: xmap is an experimental feature and probably has bugs!
  warn("xmap is an experimental feature and probably has bugs!")


key shape (8, 2)
in shape (1, 2048)
dp 1
mp 8
Total parameters: 6053381344
read from disk/gcs in 160.364s


## Run Model

Finally, we are ready to infer with the model! The first sample takes around a minute due to compilation, but after that it should only take about 10 seconds per sample.

Feel free to mess with the different sampling parameters (top_p and temp), as well as the length of the generations (gen_len, causes a recompile when changed).

You can also change other things like per_replica_batch in the previous cells to change how many generations are done in parallel. A larger batch has higher latency but higher throughput when measured in tokens generated/s. This is useful for doing things like best-of-n cherry picking.

*Tip for best results: Make sure your prompt does not have any trailing spaces, which tend to confuse the model due to the BPE tokenization used during training.*

In [ ]:
# allow text wrapping in generated output: https://stackoverflow.com/a/61401455
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
def infer(context, top_p=0.9, temp=1.0, gen_len=512):
    tokens = tokenizer.encode(context)

    provided_ctx = len(tokens)
    pad_amount = seq - provided_ctx

    padded_tokens = np.pad(tokens, ((pad_amount, 0),)).astype(np.uint32)
    batched_tokens = np.array([padded_tokens] * total_batch)
    length = np.ones(total_batch, dtype=np.uint32) * len(tokens)

    start = time.time()
    output = network.generate(batched_tokens, length, gen_len, {"top_p": np.ones(total_batch) * top_p, "temp": np.ones(total_batch) * temp})

    samples = []
    decoded_tokens = output[1][0]

    for o in decoded_tokens[:, :, 0]:
      samples.append(f"\033[1m{context}\033[0m{tokenizer.decode(o)}")

    # print(f"completion done in {time.time() - start:06}s")
    return samples

In [ ]:
#@title **Generate a new Song Title and Band Name** { form-width: "500px" }
top_p = 1 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.7 #@param {type:"slider", min:0, max:1, step:0.1}

context = """Determine the topic for these lyrics.

Line: Ah, look at all the lonely people!
Topic: Loneliness

Line: Wiping the dirt from his hands as he walks from the grave.
Topic: Death

Line: Little darling, the smiles returning to the faces.
Topic: Happiness

Line: Every breath you take.
Topic:"""

print(infer(top_p=top_p, temp=temp, gen_len=4, context=context)[0])

Determine the topic for these lyrics.

Line: Ah, look at all the lonely people!
Topic: Loneliness

Line: Wiping the dirt from his hands as he walks from the grave.
Topic: Death

Line: Little darling, the smiles returning to the faces.
Topic: Happiness

Line: Every breath you take.
Topic: Health

Line


In [ ]:
def get_topic(line):
  context = """Determine the topic for these lyrics.

Line: Ah, look at all the lonely people!
Topic: Loneliness

Line: Wiping the dirt from his hands as he walks from the grave.
Topic: Death

Line: Little darling, the smiles returning to the faces.
Topic: Happiness

Line:"""

  context += line + "\nTopic:"
  # print(context)
  result = infer(top_p=top_p, temp=temp, gen_len=4, context=context)[0]
  # print("'" + result + "'")
  result = result[len(context)+8:].strip()
  result = result.replace("Line:","")
  parts = result.split("\n")
  topic = parts[0].strip()
  return topic

topic = get_topic("Every breath you take.")
print("\n'" + topic + "'")


'Death'


In [ ]:
import random
sample_topics_filename = "/content/sample-topics.csv"
sample_topics_header = ["Line", "Topic"]

sample_topics_file = open(sample_topics_filename, 'w', newline="")
csvwriter = csv.writer(sample_topics_file)
csvwriter.writerow(sample_topics_header)

for i in range(10):
  rld = random.choice(rock_lyrics_data)
  band_name = rld[rock_lyrics_header.index("Band Name")]
  song = rld[rock_lyrics_header.index("Song")]
  print(i+1, band_name, "-", song)
  lyrics = rld[rock_lyrics_header.index("Lyrics")]
  lines = lyrics.split(". ")
  for line in lines:
    topic = get_topic(line)
    std = [line, topic]
    csvwriter.writerow(std)
    sample_topics_file.flush()

sample_topics_file.close()

1 Michael Jackson - Scream Louder (flyte Tyme Remix)
2 Neil Young - Black Coral
3 U2 - Even Better Than The Real Thing
4 Bob Dylan - House Carpenter
5 Rolling Stones - Summer Romance
6 Tom Petty - Only a Broken Heart
7 Aerosmith - The Other Side
8 The Band - Nothing Was Delivered
9 Bruce Springsteen - Roll Of The Dice
10 Van Morrison - Call My Name


In [ ]:
!gdown --id 1AF-xTo6_b5KAqPQvHe1BHlkjBHLCl93r # sample-topics-edited.csv
import csv
sample_topics_edited_file= open("sample-topics-edited.csv")
sample_topics_edited_csvreader = csv.reader(sample_topics_edited_file)
sample_topics_edited_header = next(sample_topics_edited_csvreader)
print(sample_topics_edited_header)

sample_lines = []
sample_topics = []
for row in sample_topics_edited_csvreader:
  line = row[sample_topics_edited_header.index("Line")]
  sample_lines.append(line)
  topic = row[sample_topics_edited_header.index("Topic")]
  sample_topics.append(topic)  

print("num samples", len(sample_lines))
print()

for i in range(10):
  index = random.randint(0, len(sample_lines)-1)
  print(sample_lines[index], "-", sample_topics[index])

Downloading...
From: https://drive.google.com/uc?id=1AF-xTo6_b5KAqPQvHe1BHlkjBHLCl93r
To: /content/sample-topics-edited.csv
100% 9.90k/9.90k [00:00<00:00, 4.43MB/s]
['Line', 'Topic']
num samples 263

In my soul, in my soul, in my soul - Soul
Does she know I still care? - Breakup
If skills sold - Skills
Like a sweet bird of youth - Youth
Since - Since
Lovers they walk by - Love
The hour that he passed, unwound and stopped. - Time
Walkin' on a city street who would think you could ever be touched - Bad Dreams
I'm too weak to put up a fight - Weakness
And the girls in their summer clothes - Summer


In [ ]:
def get_topic(line, samples):
  context = "Determine the topic for these lyrics.\n\n"

  for s in samples:
    context += "Line: " + s[0] + "\n"
    context += "Topic: " + s[1] + "\n\n"

  context += line + "\nTopic:"
  # print(context)
  result = infer(top_p=top_p, temp=temp, gen_len=4, context=context)[0]
  # print("'" + result + "'")
  result = result[len(context)+8:].strip()
  result = result.replace("Line:","")
  parts = result.split("\n")
  topic = parts[0].strip()
  return topic

samples = []
for i in range(3):
  index = random.randint(0, len(sample_lines)-1)
  samples.append([sample_lines[index], sample_topics[index]])

print(samples)
topic = get_topic("Every breath you take.", samples)
print("'" + topic + "'")

[['In the city of Paris and I called out your name', 'Love'], ['All your Abraham sons', 'Abraham'], ['Through my Hard Knock Life time', 'Hard Times']]
'Breath'


In [ ]:
rock_topics_filename = "/content/drive/MyDrive/public/RockTopics/rock-topics.csv"
rock_topics_header = ["Band Name", "Song", "Topics"]

rock_topics_file = open(rock_topics_filename, 'a', newline="")
csvwriter = csv.writer(rock_topics_file)
csvwriter.writerow(rock_topics_header)

for rld in rock_lyrics_data[9692:]:
  band_name = rld[rock_lyrics_header.index("Band Name")]
  song = rld[rock_lyrics_header.index("Song")]
  topic_list = []
  lyrics = rld[rock_lyrics_header.index("Lyrics")]
  lines = lyrics.split(". ")
  for line in lines:
    samples = []
    for i in range(3):
      index = random.randint(0, len(sample_lines)-1)
      samples.append([sample_lines[index], sample_topics[index]])
    topic = get_topic(line, samples)
    topic_list.append(topic)
  topics = ",".join(topic_list)
  rtd = [band_name, song, topics]
  print(rtd)
  csvwriter.writerow(rtd)
  rock_topics_file.flush()

rock_topics_file.close()

['The Beatles', 'Savoy Truffle', "Tropical fruit,Love,Coffee,The Fool,Food,Apple,Time,Life,Everything will have to,Food,Now,Love,You're going to,Sweat,Fear,Love,Flavor.,Self image,Sadness,Abnormal,Location,Montelimat,Pineapple,Good News,Body parts,Food"]
['The Beatles', "Searchin'", "Searching,Love,Love's a fool,Finding,Poverty,Searching,Since,Searching,Searching,Honey,every which way you,Police,Evil,Search,Charlie Chan, Simon,Nothing,Men and women,How many times do,She,She will see/,Walk,Music,Searching,Searching,Youth,Money,Police,Marriage,Finding a girl,Mystery,nothing,Gun,Alan B.,Searching,Self,Street,Drumming,Searching for love,Things,How to do it,Police,Love,Finding someone,Find / Catch,To find"]
['The Beatles', 'September In The Rain', 'Brown leaves,Falling down,Rain,Evening,Lights,Rain,Suffering,Love,Raindrops,Love,Spring,September,Rain,Every,Love,Rain,A melody,Spring,Time,Love,Love,Rain']
['The Beatles', 'Set Fire To That Lot!', "Topic,Loneliness,We think the show,Digging the 